In [24]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [25]:
DATA_PATH = "/content/drive/MyDrive/final_pj/final_project_data/"
DATA_PATH

'/content/drive/MyDrive/final_pj/final_project_data/'

In [26]:
SEED = 42

In [27]:
import pandas as pd
import numpy as np

train_tr = pd.read_csv(f"{DATA_PATH}store_train_transactions.csv") # 학습용 구매기록 데이터
train_target = pd.read_csv(f"{DATA_PATH}store_train.csv") # 학습용 정답 데이터
test_tr = pd.read_csv(f"{DATA_PATH}store_test_transactions.csv") # 테스트용 구매기록 데이터
submit = pd.read_csv(f"{DATA_PATH}store_submission.csv") # 제출 양식 데이터

train_tr.shape , train_target.shape , test_tr.shape , submit.shape

((523105, 7), (14940, 2), (441196, 7), (12225, 2))

In [28]:
train_ft = pd.read_csv(f"{DATA_PATH}train_common_ksj_1105.csv") # 학습 데이터(피처)
test_ft = pd.read_csv(f"{DATA_PATH}test_common_ksj_1105.csv") # 테스트 데이터(피처)

train_ft.shape , test_ft.shape

((14940, 463), (12225, 463))

In [29]:
train_ft.isnull().sum().sum()

0

In [30]:
test_ft.isnull().sum().sum()

0

In [31]:
train_ft = train_ft.iloc[:,1:]
test_ft = test_ft.iloc[:,1:]
train_ft.shape, test_ft.shape

((14940, 462), (12225, 462))

In [32]:
cols = train_ft.select_dtypes("object").columns.tolist()
train_ft[cols].nunique()

,0
주구매지점,4
주구매_중분류,246
주구매_대분류_수정,6
대분류_수정_평균금액최대,6


In [33]:
cols

['주구매지점', '주구매_중분류', '주구매_대분류_수정', '대분류_수정_평균금액최대']

In [34]:
%pip install category_encoders

In [35]:
import category_encoders as ce
enc = ce.one_hot.OneHotEncoder()
tmp = enc.fit_transform(train_ft[cols])
train_ft = pd.concat([train_ft,tmp],axis =1)

tmp = enc.transform(test_ft[cols])
test_ft = pd.concat([test_ft,tmp],axis =1)

train_ft.shape, test_ft.shape

((14940, 724), (12225, 724))

In [36]:
cols

['주구매지점', '주구매_중분류', '주구매_대분류_수정', '대분류_수정_평균금액최대']

In [37]:
train_ft = train_ft.drop(columns=cols)
test_ft = test_ft.drop(columns=cols)
train_ft.shape, test_ft.shape

((14940, 720), (12225, 720))

In [38]:
train_ft.select_dtypes("object").columns , test_ft.select_dtypes("object").columns

(Index([], dtype='object'), Index([], dtype='object'))

In [39]:
from sklearn.preprocessing import MinMaxScaler

In [40]:
scaler = MinMaxScaler()
scaler.fit(train_ft)

MinMaxScaler()

In [41]:
train_ft[train_ft.columns] = scaler.transform(train_ft)
test_ft[test_ft.columns] = scaler.transform(test_ft)
train_ft.head()

,내점일수,구매주기,주말방문비율,봄_구매비율,여름_구매비율,가을_구매비율,겨울_구매비율,주구매요일,일별평균구매건수,거래개월수,...,주구매_대분류_수정_3,주구매_대분류_수정_4,주구매_대분류_수정_5,주구매_대분류_수정_6,대분류_수정_평균금액최대_1,대분류_수정_평균금액최대_2,대분류_수정_평균금액최대_3,대분류_수정_평균금액최대_4,대분류_수정_평균금액최대_5,대분류_수정_평균금액최대_6
0,0.041494,0.130682,0.250000,0.050000,0.250000,0.400000,0.300000,0.500000,0.090909,0.545455,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,0.082988,0.090909,0.023810,0.357143,0.166667,0.357143,0.119048,0.500000,0.111111,0.909091,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,0.228216,0.034091,0.210526,0.464912,0.140351,0.175439,0.219298,0.000000,0.115079,1.000000,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,0.377593,0.017045,0.189573,0.379147,0.180095,0.236967,0.203791,0.500000,0.143720,1.000000,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,0.107884,0.062500,0.258065,0.112903,0.612903,0.209677,0.064516,0.666667,0.144033,0.818182,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [42]:
target = train_target["target"]
target

,target
0,1.0
1,1.0
2,0.0
3,0.0
4,0.0
...,...
14935,0.0
14936,0.0
14937,0.0
14938,1.0


# AutoML(Automated machine learning)
- 시간 소모적이고 반복적인 기계 학습 모델 개발 작업을 자동화하는 프로세스
- 데이터 과학자, 분석가 및 개발자는 모델 품질을 유지하면서 확장성, 효율성 및 생산성이 높은 ML 모델을 빌드할 수 있다.

In [43]:
%pip install flaml

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.9/313.9 kB 17.4 MB/s eta 0:00:00


In [46]:
%pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 5.8 MB/s eta 0:00:00


In [47]:
from flaml import AutoML
from lightgbm import LGBMClassifier

auto_ml_ens = AutoML()
params = { "metric" : "macro_f1",
           "task" : "classification",
           "time_budget" : 60*10,
           "seed" : 42,
           "early_stop" : True,
           "ensemble": {'final_estimator': LGBMClassifier(random_state=SEED)},
           "estimator_list" : ['catboost', 'lgbm', 'rf', 'xgboost', 'extra_tree', 'xgb_limitdepth']
           }
auto_ml_ens.fit(train_ft, target, **params)

[flaml.automl.logger: 11-05 06:16:16] {1728} INFO - task = classification
[flaml.automl.logger: 11-05 06:16:16] {1739} INFO - Evaluation method: holdout
[flaml.automl.logger: 11-05 06:16:17] {1838} INFO - Minimizing error metric: 1-macro_f1
[flaml.automl.logger: 11-05 06:16:17] {1955} INFO - List of ML learners in AutoML Run: ['catboost', 'lgbm', 'rf', 'xgboost', 'extra_tree', 'xgb_limitdepth']
[flaml.automl.logger: 11-05 06:16:17] {2258} INFO - iteration 0, current learner catboost
[flaml.automl.logger: 11-05 06:16:36] {2393} INFO - Estimated sufficient time budget=184032s. Estimated necessary time budget=184s.
[flaml.automl.logger: 11-05 06:16:36] {2442} INFO -  at 25.3s,	estimator catboost's best error=0.3005,	best estimator catboost's best error=0.3005
[flaml.automl.logger: 11-05 06:16:36] {2258} INFO - iteration 1, current learner lgbm
[flaml.automl.logger: 11-05 06:16:36] {2442} INFO -  at 26.0s,	estimator lgbm's best error=0.6223,	best estimator catboost's best error=0.3005
[fla

- 선택된 모델 객체 확인

In [49]:
auto_ml_ens.best_loss

0.29516615848963623

In [51]:
1-auto_ml_ens.best_loss

0.7048338415103638

In [ ]:
pred = auto_ml_ens.predict(test_ft)
pred.mean()

- 튜닝된 하이퍼파라미터

- 선택된 모델의 cv 점수 확인하기

- 예측 가능

In [ ]:
pred = auto_ml_ens.predict_proba(test_ft)[:,1]
pred.shape